Spotify for Developers/ Getting Started
documentation: https://developer.spotify.com/documentation/web-api 

- Log into dashboard using Spotify account (Facebook login for me) https://developer.spotify.com/dashboard
- Accept Spotify Developer Terms of Service. 
- Create app and select Web API
    App name: music data app https://developer.spotify.com/dashboard/ee238904262343b7abfe5e34204d3106 
    App description: data eng project app
    App Status: Development mode


In [ ]:
## Get access token
import requests
import base64

client_id = "xx"
client_secret = "xx"

# Encode client credentials
auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

# Request token
token_url = "https://accounts.spotify.com/api/token"
response = requests.post(
    token_url,
    data={"grant_type": "client_credentials"},
    headers={"Authorization": f"Basic {b64_auth_str}"}
)

print(response.status_code)
print(response.json())

token = response.json()["access_token"]
print(token[:50] + "...")  # preview first chars


200
{'access_token': 'BQCJKDvmJMUhstFoMgb_dfDmyqzDasFmLPKhsE7xBPUWmCHH1SMJE0K1KQv4qaqpkzYwpR8V0vHOKBg-5HWKbSyAVlgIULLtZcAC0bvDxj7aSc7QJaA9LH3iJhJOfeMD-5kW4E5jBUE', 'token_type': 'Bearer', 'expires_in': 3600}
BQCJKDvmJMUhstFoMgb_dfDmyqzDasFmLPKhsE7xBPUWmCHH1S...


In [ ]:
## Call API (Add token to headers)
headers = {"Authorization": f"Bearer {token}"}

# Example: search for tracks
search_url = "https://api.spotify.com/v1/search"
params = {"q": "Daft Punk", "type": "track", "limit": 5}

res = requests.get(search_url, headers=headers, params=params)
data = res.json()


In [9]:
## Preview results
for track in data["tracks"]["items"]:
    print(track["name"], "–", track["artists"][0]["name"])


Veridis Quo – Daft Punk
One More Time – Daft Punk
Get Lucky (feat. Pharrell Williams and Nile Rodgers) – Daft Punk
Starboy – The Weeknd


In [ ]:
# Search for an artist by name

search_url = "https://api.spotify.com/v1/search"
params = {"q": "Daft Punk", "type": "artist", "limit": 1}

res = requests.get(search_url, headers=headers, params=params)
artist_data = res.json()
artist = artist_data["artists"]["items"][0]

print("Name:", artist["name"])
print("ID:", artist["id"])



Name: Daft Punk
ID: 4tZwfgrHOc3mvqYlEYSvVi


In [ ]:
# Get artist details

artist_id = artist["id"]
url = f"https://api.spotify.com/v1/artists/{artist_id}"

res = requests.get(url, headers=headers)
artist_info = res.json()

print("Name:", artist_info["name"])
print("Genres:", artist_info["genres"])
print("Followers:", artist_info["followers"]["total"])


Name: Daft Punk
Genres: ['french house', 'electronic', 'electro']
Followers: 10711086


In [ ]:
# Get artist's top tracks
url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
params = {"market": "US"}  # required param

res = requests.get(url, headers=headers, params=params)
top_tracks = res.json()

for t in top_tracks["tracks"]:
    print(t["name"], "-", t["album"]["name"])


Get Lucky (Radio Edit) [feat. Pharrell Williams and Nile Rodgers] - Get Lucky (Radio Edit) [feat. Pharrell Williams and Nile Rodgers]
One More Time - Discovery
Instant Crush (feat. Julian Casablancas) - Random Access Memories
Get Lucky (feat. Pharrell Williams and Nile Rodgers) - Random Access Memories
Around the World - Homework
Harder, Better, Faster, Stronger - Discovery
Something About Us - Discovery
Veridis Quo - Discovery
Lose Yourself to Dance (feat. Pharrell Williams) - Random Access Memories
Digital Love - Discovery


In [ ]:
## Spotify API Helper with Auto Token Refresh
import requests
import base64
import time

class SpotifyClient:
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
        self.token = None
        self.token_expires = 0  # epoch time

    def _get_token(self):
        """Request a new token from Spotify"""
        auth_str = f"{self.client_id}:{self.client_secret}"
        b64_auth_str = base64.b64encode(auth_str.encode()).decode()

        token_url = "https://accounts.spotify.com/api/token"
        response = requests.post(
            token_url,
            data={"grant_type": "client_credentials"},
            headers={
                "Authorization": f"Basic {b64_auth_str}",
                "Content-Type": "application/x-www-form-urlencoded"
            }
        )
        response.raise_for_status()
        token_data = response.json()
        
        # Save token + expiry
        self.token = token_data["access_token"]
        self.token_expires = int(time.time()) + token_data["expires_in"] - 60  # 1 min safety margin

    def _ensure_token(self):
        """Refresh the token if expired"""
        if self.token is None or time.time() >= self.token_expires:
            self._get_token()

    def get(self, endpoint, params=None):
        """Call Spotify API with auto-refresh token"""
        self._ensure_token()
        url = f"https://api.spotify.com/v1/{endpoint}"
        headers = {"Authorization": f"Bearer {self.token}"}
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        return response.json()

## Usage Example

# Create client
sp = SpotifyClient("YOUR_CLIENT_ID", "YOUR_CLIENT_SECRET")

# Example: search for tracks
data = sp.get("search", params={"q": "Daft Punk", "type": "track", "limit": 5})

for track in data["tracks"]["items"]:
    print(track["name"], "–", track["artists"][0]["name"])

